In [12]:
import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

# Descargar los recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
    # Tokenización
    tokens = nltk.word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

descripcion_filtrada = df_def['Descripcion'].apply(preprocess_text)

print(descripcion_filtrada)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pjcan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


https://www.indomio.es/anuncios/95227781/    alquila piso amueblado habitaciones carrer del...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar partir ro febrero juni...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/95223135/    rincón acogedor necesito espacio gustará aquí ...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico m valencia valencia solidinmuebles vende...
https://www.indomio.es/anuncios/94958481/    venden pisos obra nueva camino moncada trata r...
https://www.indomio.es/anuncios/95006805/    vende exclusiva viviendas obra nueva calle arq...
https://www.indomio.es/anuncios/95313313/    vende piso barrio tormoseste luminoso apartame...
https://www.indomio.es/anuncios/95319871/    piso 

## LSA

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

tv = TfidfVectorizer()
svd = TruncatedSVD(n_components=100)
lsa = make_pipeline(tv, svd, Normalizer(copy=False))
lsa_topic_vectors = lsa.fit_transform(descripcion_filtrada)

lsa_topic_vectors

array([[ 1.42953276e-01, -2.65970744e-01,  7.76405164e-01, ...,
        -9.41205150e-02,  4.37982821e-02, -1.22626595e-01],
       [ 1.94808097e-01, -2.00928641e-01,  3.40684548e-02, ...,
        -4.97638653e-02,  4.43798128e-03, -7.47810012e-02],
       [ 2.84442178e-01, -3.06021405e-01, -8.11883828e-02, ...,
         8.07207247e-02,  1.71463886e-01,  4.54825644e-02],
       ...,
       [ 2.55388741e-01, -2.32356896e-01, -6.18326994e-02, ...,
        -4.65383532e-02,  3.14691332e-02,  6.36793096e-02],
       [ 3.99301157e-01, -3.86111892e-01,  3.16656123e-03, ...,
         4.48207025e-02,  6.96099914e-03, -2.87383854e-04],
       [ 3.14087587e-01, -3.32400476e-01,  1.43689610e-02, ...,
        -4.96452898e-02,  4.73212721e-02, -1.87283749e-02]])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción usando la misma pipeline LSA
vector_descripcion = lsa.transform([descripcion])

# Calcular la similitud coseno entre la nueva descripción y todas las descripciones existentes
similarities = cosine_similarity(vector_descripcion, lsa_topic_vectors)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Opcionalmente, puedes mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()


Descripción: DISPONIBLE A PARTIR DE JULIO 2024. ALQUILER PARA ESTUDIANTES UNIVERSITARIOS (CONTRATO DE TEMPORADA). Se alquila este precioso apartamento , semireformado, en un edificio tranquilo y en una zona ideal si te gusta tener todo a un paso. Se trata de una vivienda con un total de 4 habitaciones y dos cuartos de baños completos. el apartamento se distribuye en zona de salón comedor con acceso a balcón y aire acondicionado (frío /calor), 4 habitaciones con camas de 1,35, totalmente equipadas y dos cuartos de baños completos. Cocina independiente, con todo lo que puedas necesitar en tu día a día y con acceso directo a terraza en zona común, pero de uso privativo. Como ves el piso lo tiene todo, es bonito, esta cerca de transporte público, tanto metro como autobús en menos de 9 minutos a pie, 5 minutos andando de supermercado y el mercado municipal del Cabañal a 7 minutos y lo mejor de todo, a tan solo 16 minutos a pie de la playa. No se puede pedir más. Si estás buscando una vivien

## LDA

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)

tf = tf_vectorizer.fit_transform(descripcion_filtrada)

lda = LatentDirichletAllocation(n_components=10, max_iter=5,
learning_method='online',
learning_offset=50.,
random_state=0)

lda_topic_matrix = lda.fit_transform(tf)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

# Nueva descripción
descripcion = "Quiero un piso de unos 100 m2 que este bien iluminado, cueste menos de 700 euros al mes, este en la zona de campanar y tenga por lo menos 3 habitaciones y 2 baños"

# Transformar la nueva descripción
vector_descripcion = tf_vectorizer.transform([descripcion])
vector_descripcion_lda = lda.transform(vector_descripcion)

# Calcular la similitud coseno entre la nueva descripción y las descripciones existentes
similarities = cosine_similarity(vector_descripcion_lda, lda_topic_matrix)

# Obtener los índices de las descripciones más similares en orden descendente
most_similar_indices = similarities.argsort()[0][::-1]

# Mostrar las descripciones más similares y sus similitudes
for index in most_similar_indices[:5]:  # muestra las 5 descripciones más similares
    print(f"Descripción: {df_def['Descripcion'].iloc[index]}")
    print(f"Similitud: {similarities[0][index]}")
    print()

Descripción: Bonito piso recién reformado. Está todo a estrenar! A tan solo unos minutos de la plaza del Ayuntamienti y del Mercado Central. 
Similitud: 0.9999964260327223

Descripción: Piso en pleno centro de Valencia de 545 m2 con terrazas muy grandes. Dispone de un apartamento añadido para los invitados.el piso es de estilo señorial con acabados de primera calidad reformado y a estrenar.
Similitud: 0.9999261534660971

Descripción: Apartamento de 3 dormitorios en una excelente ubicación, Restaurantes, supermercados cercas.El dormitorio principal tiene un vestidor y un baño en suite. La cocina está amueblada con área de lavandería separada. Aire acondicionado
Similitud: 0.9999139048645552

Descripción: Un encantador apartamento con lindas paredes azules en la sala de estar, una espaciosa cocina y una gran terraza en un vecindario tranquilo y residencial. El apartamento está a solo 10 minutos a pie del puerto de Valencia, y cerca encontrará algunas tiendas, una piscina municipal y una 